In [ ]:
import numpy as np
import math
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS

from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume, Returns, RSI
from quantopian.interactive.data.sentdex import sentiment
from quantopian.pipeline.data.sentdex import sentiment
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data.zacks import EarningsSurprises

import talib
import pandas as pd
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics

bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios

In [ ]:
class LowVol(CustomFactor):
    inputs = [Returns(window_length=2)]
    window_length = 25
    
    def compute(self, today, assets, out, close):
        out[:] = -np.nanstd(close, axis=0)

In [ ]:
def make_pipeline():
    #LowVol, Revenue Growth, Sentiment
    universe = QTradableStocksUS()
    
    #use factors[] then loop throughc
    
    testing_factor1 = LowVol(mask=universe)
    testing_factor2 = or_.revenue_growth.latest
    testing_factor3 = sentiment.sentiment_signal.latest

    universe = (QTradableStocksUS() 
                & testing_factor1.notnull()
                & testing_factor2.notnull()
                & testing_factor3.notnull())
    
    testing_factor1 = testing_factor1.rank(mask=universe, method='average')
    testing_factor2 = testing_factor2.rank(mask=universe, method='average')
    testing_factor3 = testing_factor3.rank(mask=universe, method='average')
    
    testing_factor = testing_factor1 + testing_factor2 + testing_factor3
    
    testing_quantiles = testing_factor.quantiles(2)
    
    pipe = Pipeline(columns={
        'testing_factor': testing_factor,
        'shorts':testing_quantiles.eq(0),
        'longs':testing_quantiles.eq(1),
    },screen= universe)
    
    return pipe

result = run_pipeline(make_pipeline(), start_date='2015-01-01',end_date='2016-01-01')
result.head()
    

In [ ]:
assets = result.index.levels[1].unique()
len(assets)

In [ ]:
pricing_data = get_pricing(assets,start_date='2014-12-01',end_date='2016-02-01', fields='open_price')

In [ ]:
import alphalens as al
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_returns_tear_sheet

factor_data = get_clean_factor_and_forward_returns(
    result['testing_factor'], # How to analyze a specific pipeline column with Alphalens
    pricing_data,
    quantiles = 2,
    periods=(1,5,10)
)

create_returns_tear_sheet(factor_data=factor_data)